## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-10-52-06 +0000,nypost,Ghislaine Maxwell told DOJ Trump never did any...,https://nypost.com/2025/08/06/us-news/ghislain...
1,2025-08-06-10-50-06 +0000,wapo,A Danish zoo is seeking unwanted pets — to fee...,https://www.washingtonpost.com/world/2025/08/0...
2,2025-08-06-10-45-08 +0000,nyt,NFL Will Get a Stake in ESPN in a Complex Deal,https://www.nytimes.com/2025/08/05/business/me...
3,2025-08-06-10-41-00 +0000,latimes,"Disney's streaming business continues to grow,...",https://www.latimes.com/entertainment-arts/bus...
4,2025-08-06-10-41-00 +0000,wsj,"Disney Looks to Higher Streaming, Parks Growth",https://www.wsj.com/business/earnings/disney-d...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,55
298,war,14
442,gaza,12
213,new,12
121,out,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
113,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,117
57,2025-08-06-09-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...,111
9,2025-08-06-10-11-50 +0000,nypost,Trump envoy Steve Witkoff meets Putin ahead of...,https://nypost.com/2025/08/06/world-news/trump...,104
232,2025-08-05-19-42-59 +0000,nypost,Zelensky says Ukraine ready to ink drone deal ...,https://nypost.com/2025/08/05/us-news/zelensky...,101
176,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...,99


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
113,117,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...
330,57,2025-08-05-13-39-25 +0000,nypost,CEO ignored ‘red flags’ ahead of ‘preventable’...,https://nypost.com/2025/08/05/world-news/titan...
57,53,2025-08-06-09-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
21,50,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
26,47,2025-08-06-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Death row inmate cr...,https://nypost.com/2025/08/06/us-news/ny-postc...
295,43,2025-08-05-16-34-00 +0000,wsj,The Swiss president is heading to the U.S. in ...,https://www.wsj.com/economy/trade/swiss-presid...
80,42,2025-08-06-05-41-24 +0000,nypost,Long Island fire chief Peter Alt caught cursin...,https://nypost.com/2025/08/06/us-news/long-isl...
287,38,2025-08-05-16-53-34 +0000,nypost,Former X CEO Linda Yaccarino lands new gig at ...,https://nypost.com/2025/08/05/business/former-...
331,38,2025-08-05-13-27-00 +0000,cbc,"India, Russia hit back after U.S. accuses Modi...",https://www.cbc.ca/news/world/india-us-russia-...
165,37,2025-08-05-22-53-49 +0000,nypost,Netanyahu believes occupying Gaza is the only ...,https://nypost.com/2025/08/05/world-news/netan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
